In [2]:
#load libraries
import os
import glob
from eumap import parallel
from eumap.parallel import TilingProcessing
from eumap.raster import read_rasters, save_rasters
from eumap.misc import ttprint
from eumap.misc import find_files
import re
import pandas as pd
import numpy as np
import rasterio
from pathlib import Path
from osgeo import gdal
import geopandas as gpd
import matplotlib.pyplot as plt
import random
from shapely.geometry import Polygon, Point
from eumap.mapper import SpaceOverlay
from random import sample
import joblib

In [37]:
# load landmask and save as joblib instance
base_raster = ["../nightlights.average_viirs.v21_m_250m_s_20210101_20211231_go_epsg4326_v20230318.tif"]
base_ds = rasterio.open(base_raster[0])

In [2]:
#create raster where all 4 human pressure variables are zero at 2000 and 2021
#load in 1st dataset
nl_2000_path = ["../nightlights.average_viirs.v21_m_250m_s_20000101_20001231_go_epsg4326_v20230318.tif"]
ds_nl_2000 = rasterio.open(nl_2000_path[0])
nl_2000_data = rasterio.open(nl_2000_path[0]).read(1)

hp_zero = np.zeros_like(nl_2000_data)
hp_zero[(nl_2000_data > 0)] = 100

In [3]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del nl_2000_data
gc.collect()

nl_2021_path = ["../nightlights.average_viirs.v21_m_250m_s_20210101_20211231_go_epsg4326_v20230318.tif"]
nl_2021_data = rasterio.open(nl_2021_path[0]).read(1)
hp_zero[(nl_2021_data > 0)] = 100

In [4]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del nl_2021_data
gc.collect()

hfp_2021_path = ["../lcv_human.footprint_hpressure.50_m_250m_epsg4326_2021_epsg4326_v1.0.tif"]
hfp_2021_data = rasterio.open(hfp_2021_path[0]).read(1)
hp_zero[(hfp_2021_data > 0)] = 100

In [5]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del hfp_2021_data
gc.collect()

hfp_2000_path = ["../lcv_human.footprint_hpressure.50_m_250m_epsg4326_2000_epsg4326_v1.0.tif"]
hfp_2000_data = rasterio.open(hfp_2000_path[0]).read(1)
hp_zero[(hfp_2000_data > 0)] = 100

In [6]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del hfp_2000_data
gc.collect()

crop_2000_path = ["../lcv_globalcropland_bowen.et.al_p_250m_epsg4326_s0..0cm_2000_v0.1.tif"]
crop_2000_data = rasterio.open(crop_2000_path[0]).read(1)
hp_zero[(crop_2000_data > 0)] = 100

In [7]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del crop_2000_data
gc.collect()

crop_2021_path = ["../lcv_globalcropland_bowen.et.al_p_250m_epsg4326_s0..0cm_2021_v0.1.tif"]
crop_2021_data = rasterio.open(crop_2021_path[0]).read(1)
hp_zero[(crop_2021_data > 0)] = 100

In [8]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del crop_2021_data
gc.collect()

pop_2021_path = ["../pop.count_ghs.jrc_m_250m_s_20210101_20211231_go_epsg.4326_v20230620.tif"]
pop_2021_data = rasterio.open(pop_2021_path[0]).read(1)
hp_zero[(pop_2021_data > 0)] = 100

In [9]:
#garbage 1st dataset, and load in 2nd dataset
import gc
del pop_2021_data
gc.collect()

pop_2000_path = ["../pop.count_ghs.jrc_m_250m_s_20000101_20001231_go_epsg.4326_v20230620.tif"]
pop_2000_data = rasterio.open(pop_2000_path[0]).read(1)
hp_zero[(pop_2000_data > 0)] = 100

In [19]:
#write to file
# with rasterio.open(
#     'hp.ind_v20230628.tif', 
#     'w',
#     driver='GTiff',
#     height=hp_zero.shape[0],
#     width=hp_zero.shape[1],
#     count=1,
#     dtype=np.int16,
#     crs=base_ds.crs,
#     transform=base_ds.transform,
#     compress='lzw',
#     tiled=True
# ) as dest_file:
#     dest_file.write(hp_zero, 1)
# dest_file.close()

In [ ]:
input_vrt = 'hp.ind_v20230628.tif'
output_vrt = input_vrt.replace(".tif", "_cog.tif")
cmd = f'gdal_translate --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {input_vrt} {output_vrt}'
# os.system(cmd)

In [ ]:
#make lc change map

forest_2000_path = ["../forest.ind_esa.cci.lc_pc_250m_s_20000101_20001231_go_epsg4326_v20230627.tif"]
forest_2000_data = rasterio.open(forest_2000_path[0]).read(1)

forest_2001_path = ["../forest.ind_esa.cci.lc_pc_250m_s_20010101_20011231_go_epsg4326_v20230627.tif"]
forest_2001_data = rasterio.open(forest_2001_path[0]).read(1)

wet_2000_path = ["../wetland.ind_esa.cci.lc_pc_250m_s_20000101_20001231_go_epsg4326_v20230627.tif"]
wet_2000_data = rasterio.open(wet_2000_path[0]).read(1)

wet_2001_path = ["../wetland.ind_esa.cci.lc_pc_250m_s_20010101_20011231_go_epsg4326_v20230627.tif"]
wet_2001_data = rasterio.open(wet_2001_path[0]).read(1)



In [ ]:
crop_2019_path = ["../cropland.ind_esa.cci.lc_pc_250m_s_20190101_20191231_go_epsg4326_v20230627.tif"]
crop_2019_data = rasterio.open(crop_2019_path[0]).read(1)

crop_2020_path = ["../cropland.ind_esa.cci.lc_pc_250m_s_20200101_20201231_go_epsg4326_v20230627.tif"]
crop_2020_data = rasterio.open(crop_2020_path[0]).read(1)

grass_2019_path = ["../grassland.ind_esa.cci.lc_pc_250m_s_20190101_20191231_go_epsg4326_v20230627.tif"]
grass_2019_data = rasterio.open(grass_2019_path[0]).read(1)

grass_2020_path = ["../grassland.ind_esa.cci.lc_pc_250m_s_20200101_20201231_go_epsg4326_v20230627.tif"]
grass_2020_data = rasterio.open(grass_2020_path[0]).read(1)



In [39]:
crop_2000_path = ["../cropland.ind_esa.cci.lc_pc_250m_s_20000101_20001231_go_epsg4326_v20230627.tif"]
crop_2000_data = rasterio.open(crop_2000_path[0]).read(1)

crop_2001_path = ["../cropland.ind_esa.cci.lc_pc_250m_s_20010101_20011231_go_epsg4326_v20230627.tif"]
crop_2001_data = rasterio.open(crop_2001_path[0]).read(1)

grass_2000_path = ["../grassland.ind_esa.cci.lc_pc_250m_s_20000101_20001231_go_epsg4326_v20230627.tif"]
grass_2000_data = rasterio.open(grass_2000_path[0]).read(1)

grass_2001_path = ["../grassland.ind_esa.cci.lc_pc_250m_s_20010101_20011231_go_epsg4326_v20230627.tif"]
grass_2001_data = rasterio.open(grass_2001_path[0]).read(1)

forest_2019_path = ["../forest.ind_esa.cci.lc_pc_250m_s_20190101_20191231_go_epsg4326_v20230627.tif"]
forest_2019_data = rasterio.open(forest_2019_path[0]).read(1)

forest_2020_path = ["../forest.ind_esa.cci.lc_pc_250m_s_20200101_20201231_go_epsg4326_v20230627.tif"]
forest_2020_data = rasterio.open(forest_2020_path[0]).read(1)

wet_2019_path = ["../wetland.ind_esa.cci.lc_pc_250m_s_20190101_20191231_go_epsg4326_v20230627.tif"]
wet_2019_data = rasterio.open(wet_2019_path[0]).read(1)

wet_2020_path = ["../wetland.ind_esa.cci.lc_pc_250m_s_20200101_20201231_go_epsg4326_v20230627.tif"]
wet_2020_data = rasterio.open(wet_2020_path[0]).read(1)

In [40]:
lc_change_neg = np.zeros_like(forest_2000_data)
lc_change_neg[(((forest_2000_data == 100) & (forest_2001_data == 100)) | ((wet_2000_data == 100) & (wet_2001_data == 100))) & (((crop_2019_data == 100) & (crop_2020_data == 100)) | ((grass_2019_data == 100) & (grass_2020_data == 100)))] = 1
lc_change_neg[(((forest_2019_data == 100) & (forest_2020_data == 100)) | ((wet_2019_data == 100) & (wet_2020_data == 100))) & (((crop_2000_data == 100) & (crop_2001_data == 100)) | ((grass_2000_data == 100) & (grass_2001_data == 100)))] = 2


In [49]:
# ds_forest_2000 = rasterio.open(forest_2000_path[0])
# #write to file
# with rasterio.open(
#     'lc.change.ind_v20230628.tif', 
#     'w',
#     driver='GTiff',
#     height=lc_change_neg.shape[0],
#     width=lc_change_neg.shape[1],
#     count=1,
#     dtype=np.uint8,
#     crs=ds_forest_2000.crs,
#     transform=ds_forest_2000.transform,
#     compress='lzw',
#     tiled=True
# ) as dest_file:
#     dest_file.write(lc_change_neg, 1)
# dest_file.close()

In [ ]:
import gc
del forest_2019_data, forest_2020_data, wet_2019_data, wet_2020_data
gc.collect()


In [ ]:
import os
input_vrt = 'lc.change.ind_v20230628.tif'
output_vrt = input_vrt.replace(".tif", "_cog.tif")
cmd = f'gdal_translate --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {input_vrt} {output_vrt}'
# os.system(cmd)

In [54]:
#create 250m COG with bbox of OEMC of difference raster nightlights 2000 - 2021
cmd = 'gdalwarp -te -180.00000 -62.00081 179.99994 87.37000 -tr 0.002083333 0.002083333 -r cubicspline --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG /mnt/freya/julia/potential_fapar/data/nightlights/avg_change/nightlights.difference_viirs.v21_m_500m_s_2000_2021_go_epsg4326_v20230318.tif /mnt/freya/julia/potential_fapar/data/nightlights/avg_change/nightlights.difference_viirs.v21_m_250m_s_2000_2021_go_epsg4326_v20230318_cog.tif'
# os.system(cmd)

Creating output file that is 172800P x 71698L.
Using internal nodata values (e.g. -32768) for image /mnt/freya/julia/potential_fapar/data/nightlights/avg_change/nightlights.difference_viirs.v21_m_500m_s_2000_2021_go_epsg4326_v20230318.tif.
Copying nodata values from source /mnt/freya/julia/potential_fapar/data/nightlights/avg_change/nightlights.difference_viirs.v21_m_500m_s_2000_2021_go_epsg4326_v20230318.tif to destination .
0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [3]:
file_path = ['../nightlights.difference_viirs.v21_m_250m_s_2000_2021_go_epsg4326_v20230318_cog.tif']
nl_diff_data = rasterio.open(file_path[0]).read(1)

In [40]:
#create new nl diff data stratum

#load in 1st dataset
nl_2000_path = ["../nightlights.average_viirs.v21_m_250m_s_20000101_20001231_go_epsg4326_v20230318.tif"]
ds_nl_2000 = rasterio.open(nl_2000_path[0])
nl_2000_data = rasterio.open(nl_2000_path[0]).read(1)

In [ ]:
nl_2021_path = ["../nightlights.average_viirs.v21_m_250m_s_20210101_20211231_go_epsg4326_v20230318.tif"]
nl_2021_data = rasterio.open(nl_2021_path[0]).read(1)


In [19]:
nl_diff_data_pos = nl_diff_data[nl_diff_data>0]

In [32]:
nl_diff_data_neg = nl_diff_data[nl_diff_data<0]

In [39]:
base_raster = ["nightlights.average_viirs.v21_m_250m_s_20210101_20211231_go_epsg4326_v20230318.tif"]
nl_data_2021 = rasterio.open(base_raster[0]).read(1)

In [43]:
nl_data_2021_notzero = nl_data_2021[nl_data_2021>0]

In [50]:
nl_data_2021_neg = nl_data_2021[nl_data_2021<0]

In [105]:
strata = np.zeros_like(nl_diff_data)

In [62]:
lc_change_path = ["lc.change.ind_v20230628.tif"]
lc_change_data = rasterio.open(lc_change_path[0]).read(1)

In [65]:
hp4_path = ["hp.ind_v20230628.tif"]
hp4_data = rasterio.open(hp4_path[0]).read(1)

In [67]:
landmask_path = ["lcv_landmask_esacci.lc.l4_c_250m_s0..0cm_2000..2015_v1.0.tif"]
landmask_data = rasterio.open(landmask_path[0]).read(1)
landmask_data.shape

(71698, 172800)

In [107]:
#create strata
#1
strata[(nl_diff_data<10) & (nl_diff_data>-10) & (lc_change_data==0) & (hp4_data==100)] = 1

#2
strata[(nl_diff_data<10) & (nl_diff_data>-10)& (lc_change_data==1) & (hp4_data==0)] = 2

#3
strata[(nl_diff_data<10) & (nl_diff_data>-10) & (lc_change_data==1) & (hp4_data==100)] = 3

#4
strata[(nl_diff_data<10) & (nl_diff_data>-10) & (lc_change_data==2) & (hp4_data==0)] = 4

#5
strata[(nl_diff_data<10) & (nl_diff_data>-10)& (lc_change_data==2) & (hp4_data==100)] = 5

#6
strata[(nl_diff_data>10) & (lc_change_data==0) & (hp4_data==0)] = 6

#7
strata[(nl_diff_data>10) & (lc_change_data==0) & (hp4_data==100)] = 7

#8
strata[(nl_diff_data>10) & (lc_change_data==1) & (hp4_data==0)] = 8

#9
strata[(nl_diff_data>10) & (lc_change_data==1) & (hp4_data==100)] = 9

#10
strata[(nl_diff_data>10) & (lc_change_data==2) & (hp4_data==0)] = 10

#11
strata[(nl_diff_data>10) & (lc_change_data==2) & (hp4_data==100)] = 11

#12
strata[(nl_diff_data<-10) & (lc_change_data==0) & (hp4_data==0)] = 12

#13
strata[(nl_diff_data<-10) & (lc_change_data==0) & (hp4_data==100)] = 13

#14
strata[(nl_diff_data<-10) & (lc_change_data==1) & (hp4_data==0)] = 14

#15
strata[(nl_diff_data<-10) & (lc_change_data==1) & (hp4_data==100)] = 15

#16
strata[(nl_diff_data<-10) & (lc_change_data==2) & (hp4_data==0)] = 16

#17
strata[(nl_diff_data<-10) & (lc_change_data==2) & (hp4_data==100)] = 17

#18
strata[(nl_diff_data<10) & (nl_diff_data>-10) & (lc_change_data==0) & (hp4_data==0)] = 18

#landmask
strata[(landmask_data == 2) | (landmask_data==4)] = 0

In [109]:
#get unique values and counts of each value
unique, counts = np.unique(strata, return_counts=True)

#display unique values and counts side by side
print(np.asarray((unique, counts)).T)

[[         0 9110768134]
 [         1 2245165086]
 [         2    2072589]
 [         3   15846791]
 [         4    1915900]
 [         5   15056032]
 [         6      23327]
 [         7   50149043]
 [         8          2]
 [         9     318161]
 [        10         30]
 [        11     253155]
 [        13   27420619]
 [        15     197711]
 [        17     124282]
 [        18  920103538]]


In [80]:
landmask_path = ["lcv_landmask_esacci.lc.l4_c_250m_s0..0cm_2000..2015_v1.0.tif"]
landmask_ds = rasterio.open(landmask_path[0])

In [114]:
# #write to file
# with rasterio.open(
#     'strata.human.pressure.tif', 
#     'w',
#     driver='GTiff',
#     height=strata.shape[0],
#     width=strata.shape[1],
#     count=1,
#     dtype=np.int16,
#     crs=landmask_ds.crs,
#     transform=landmask_ds.transform,
#     compress='lzw',
#     tiled=True
# ) as dest_file:
#     dest_file.write(strata, 1)
# dest_file.close()

In [ ]:
input_vrt = 'strata.human.pressure.tif'
output_vrt = input_vrt.replace(".tif", "_cog.tif")
cmd = f'gdal_translate --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {input_vrt} {output_vrt}'
# os.system(cmd)

In [123]:
# reproject to equal distance projection
input_vrt = 'strata.human.pressure_cog.tif'
output_vrt = input_vrt.replace(".tif", "epsg54052.tif")
cmd = f"gdalwarp -t_srs '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs' --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {input_vrt} {output_vrt}"
# os.system(cmd)

Creating output file that is 220833P x 84474L.
0...10...20...30...40...50...60...70...80...90...100 - done.


0

1. reproject FAPAR Jan, Apr, Jul, Oct 2001 and 2021 to IGH 
2. read in FAPAR IGH dates (8 layers)
3. read in hp_strata IGH 
4. create new array for each stratum with FAPAR values in that stratum and everything else None
5. calc st.dev for each stratum array
6. calculate strata size for each stratum array
7. fill in Neyman allocation formula

In [ ]:
# 1. Reproject FAPAR p50 layers to IGH (equal distance projection)

import os
input_files_ls = [
    '../fapar_essd.lstm_p50_250m_s_20010101_20010131_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20010401_20010430_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20010701_20010731_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20011001_20011031_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20210101_20210131_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20210401_20210430_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20210701_20210731_go_epsg.4326_v20230628.tif',
    '../fapar_essd.lstm_p50_250m_s_20211001_20211031_go_epsg.4326_v20230628.tif'
]


import os
from eumap import parallel
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
def worker(cmds):
    print(f'{cmds}')
    os.system(cmds)
    return f'Command {cmds} finished'


cmd_ls = []
out_folder = '../fapar/'
for i in input_files_ls:
    output_file = out_folder + i.split('/')[-1].replace("4326_v20230628", "54052_v20230821")
    cmd = f"gdalwarp -t_srs '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs' --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {i} {output_file}"
    cmd_ls.append(cmd)

cmd_ls_nest = [[i] for i in cmd_ls]
len(cmd_ls_nest)

for result in parallel.job(worker, cmd_ls_nest, n_jobs=-10):
    print(result)

In [2]:
# 2. read in FAPAR IGH 
fapar_input_ls = [
    '../fapar/fapar_essd.lstm_p50_250m_s_20010101_20010131_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20010401_20010430_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20010701_20010731_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20011001_20011031_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20210101_20210131_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20210401_20210430_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20210701_20210731_go_epsg.54052_v20230821.tif',
    '../fapar/fapar_essd.lstm_p50_250m_s_20211001_20211031_go_epsg.54052_v20230821.tif'
]


fapar_igh_ds = rasterio.open(fapar_input_ls[0])
fapar_igh_data_1 = rasterio.open(fapar_input_ls[0]).read(1)
fapar_igh_data_2 = rasterio.open(fapar_input_ls[1]).read(1)
fapar_igh_data_3 = rasterio.open(fapar_input_ls[2]).read(1)
fapar_igh_data_4 = rasterio.open(fapar_input_ls[3]).read(1)
fapar_igh_data_5 = rasterio.open(fapar_input_ls[4]).read(1)
fapar_igh_data_6 = rasterio.open(fapar_input_ls[5]).read(1)
fapar_igh_data_7 = rasterio.open(fapar_input_ls[6]).read(1)
fapar_igh_data_8 = rasterio.open(fapar_input_ls[7]).read(1)
fapar_igh_data_8.shape

(84474, 220834)

In [3]:
fapar_data_ls = [fapar_igh_data_1, fapar_igh_data_2, fapar_igh_data_3, fapar_igh_data_4, fapar_igh_data_5, fapar_igh_data_6, fapar_igh_data_7, fapar_igh_data_8]
# fapar_igh_data_stack = np.stack(fapar_data_ls, axis=2)

In [ ]:
# import gc
# del fapar_igh_data_1, fapar_igh_data_2, fapar_igh_data_3, fapar_igh_data_4, fapar_igh_data_5, fapar_igh_data_6, fapar_igh_data_7, fapar_igh_data_8
# gc.collect()

# import gc
# del fapar_data_ls
# gc.collect()

In [2]:
# 3. read in hp strata IGH
hp_layer = ['strata.human.pressure_cog_epsg54052.tif']
hp_igh_ds = rasterio.open(hp_layer[0])
hp_igh_data = rasterio.open(hp_layer[0]).read(1)
hp_igh_data.shape

(84474, 220834)

In [ ]:
# reproject DTM to IGH and exactly FAPAR extent, and mask out values in hp_igh_data where FAPAR or DTM have no value

#DTM elevation path
elev_path = 'dtm_elevation_merit.dem_m_250m_s0..0cm_2017_v1.0.tif'
outpath =  'dtm_elevation_merit.dem_m_250m_s0..0cm_2017_v1.0_epsg.54052_v20230821.tif'

#reproject to IGH
cmd = f"gdalwarp -t_srs '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs' --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {elev_path} {outpath}"
# os.system(cmd)

dtm_fapar_extent_path = 'dtm_elevation_merit.dem_m_250m_s0..0cm_2017_v1.0_epsg.54052_v20230821_exactlyFAPARext.tif'
#warp to exactly hp_igh data extent to filter out data gaps in training points
cmd = f'gdalwarp -te -20037313.5386688336730003 -6728829.4338057115674019 20037116.4960369616746902 8600546.4899729527533054 -tr 181.4685693086472043 181.4685693086472043 -r nearest --config GDAL_CACHEMAX 2048 -co BLOCKSIZE=1024 -co BIGTIFF=YES -co COMPRESS=DEFLATE -co NUM_THREADS=8 -co LEVEL=9 -of COG {outpath} {dtm_fapar_extent_path}'
# os.system(cmd)

In [29]:
#read in DTM and substitute values in hp_strata with -9999 where DTM = -9999 or FAPAR = 255
dtm_data = rasterio.open(dtm_fapar_extent_path).read(1)
print(dtm_data.shape)
print(hp_igh_data.shape)

(84474, 220834)
(84474, 220834)


In [35]:
hp_igh_data[dtm_data==-9999]=-9999
hp_igh_data[fapar_igh_data_1==255]=-9999

In [ ]:
strata_ls = list(range(1,6))  + [7] + [9] + list(range(11, 18))
rest_stratum = [6, 8, 10, 18]


std_ls =[]
strata_size_ls = []

for stratum in strata_ls:
    print(stratum)
    fapar_igh_data_1_stratum = fapar_igh_data_1[hp_igh_data==stratum]
    print(fapar_igh_data_1_stratum.size)
   
    fapar_igh_data_2_stratum = fapar_igh_data_2[hp_igh_data==stratum]
    print(fapar_igh_data_2_stratum.size)
    fapar_igh_data_3_stratum = fapar_igh_data_3[hp_igh_data==stratum]
    fapar_igh_data_4_stratum = fapar_igh_data_4[hp_igh_data==stratum]
    fapar_igh_data_5_stratum = fapar_igh_data_5[hp_igh_data==stratum]
    fapar_igh_data_6_stratum = fapar_igh_data_6[hp_igh_data==stratum]
    fapar_igh_data_7_stratum = fapar_igh_data_7[hp_igh_data==stratum]
    fapar_igh_data_8_stratum = fapar_igh_data_8[hp_igh_data==stratum]
    
    A = [
        fapar_igh_data_1_stratum, 
        fapar_igh_data_2_stratum, 
        fapar_igh_data_3_stratum, 
        fapar_igh_data_4_stratum,
        fapar_igh_data_5_stratum,
        fapar_igh_data_6_stratum,
        fapar_igh_data_7_stratum,
        fapar_igh_data_8_stratum
] 
    print("calc N")
    N = float(sum([i.size for i in A]))
    mean_ = sum([i.sum() for i in A])/N
    std_ = np.sqrt(sum([((i-mean_)**2).sum() for i in A])/N)
    print(std_)
    std_ls.append(std_)
    strata_size_ls.append(fapar_igh_data_1_stratum.size*8) # adds total stratum size for each of the strata in hp_data (but times 8 because of 8 dates)


In [ ]:
# add rest stratum to list

fapar_igh_data_1_stratum = fapar_igh_data_1[np.isin(hp_igh_data, rest_stratum)]
print(fapar_igh_data_1_stratum.size)

fapar_igh_data_2_stratum = fapar_igh_data_2[np.isin(hp_igh_data, rest_stratum)]
print(fapar_igh_data_2_stratum.size)

fapar_igh_data_3_stratum = fapar_igh_data_3[np.isin(hp_igh_data, rest_stratum)]
fapar_igh_data_4_stratum = fapar_igh_data_4[np.isin(hp_igh_data, rest_stratum)]
fapar_igh_data_5_stratum = fapar_igh_data_5[np.isin(hp_igh_data, rest_stratum)]
fapar_igh_data_6_stratum = fapar_igh_data_6[np.isin(hp_igh_data, rest_stratum)]
fapar_igh_data_7_stratum = fapar_igh_data_7[np.isin(hp_igh_data, rest_stratum)]
fapar_igh_data_8_stratum = fapar_igh_data_8[np.isin(hp_igh_data, rest_stratum)]
    
A = [
    fapar_igh_data_1_stratum, 
    fapar_igh_data_2_stratum, 
    fapar_igh_data_3_stratum, 
    fapar_igh_data_4_stratum,
    fapar_igh_data_5_stratum,
    fapar_igh_data_6_stratum,
    fapar_igh_data_7_stratum,
    fapar_igh_data_8_stratum
] 
print("calc N")
N = float(sum([i.size for i in A]))
mean_ = sum([i.sum() for i in A])/N
std_ = np.sqrt(sum([((i-mean_)**2).sum() for i in A])/N)
print(std_)
std_ls.append(std_)
strata_size_ls.append(fapar_igh_data_1_stratum.size*8)

In [47]:
std_df = pd.DataFrame(
    {'stratum': list(range(1,6)) + [7] + [9] + list(range(11, 19)),
     'size': strata_size_ls,
     'std': std_ls
    })

In [50]:
std_df_nona2 = std_df.dropna()

In [ ]:
#calculate sample size per stratum
total_sample_size_to_allocate = 12500-(200*len(std_df_nona2))
total_sample_size_to_allocate
allstr_val = sum(x * y for x, y in zip(list(std_df_nona2['std']), list(std_df_nona2['size'])))
std_df_nona2['n_samples_incl_min200'] = round(((total_sample_size_to_allocate * (std_df_nona2['size'] * std_df_nona2['std']) / allstr_val)+200),0)


In [7]:
#subset hp_igh data again to dtm and fapar valid values
dtm_fapar_extent_path = 'dtm_elevation_merit.dem_m_250m_s0..0cm_2017_v1.0_epsg.54052_v20230821_exactlyFAPARext.tif'

dtm_data = rasterio.open(dtm_fapar_extent_path).read(1)

fapar_igh_data_1 = rasterio.open(fapar_input_ls[0]).read(1)

print(dtm_data.shape)
print(hp_igh_data.shape)

hp_igh_data[dtm_data==-9999]=-9999
hp_igh_data[fapar_igh_data_1==255]=-9999


(84474, 220834)
(84474, 220834)


In [9]:
# import gc
# del dtm_data, fapar_igh_data_1
# gc.collect()

0

In [3]:
#draw random samples from each stratum of given size
lon = np.arange(0.5, hp_igh_ds.width + 0.5, dtype='float32')
lat = np.arange(0.5, hp_igh_ds.height + 0.5, dtype='float32')
grid = np.meshgrid(lon, lat)

In [10]:
lon_grid, lat_grid = hp_igh_ds.transform * grid
lat_grid.shape, lon_grid.shape

((84474, 220834), (84474, 220834))

In [11]:
# import gc
# del grid
# gc.collect()

0

In [19]:
#assign 18 to the rest_stratum values
rest_stratum = [6, 8, 10, 18]
hp_igh_data[np.isin(hp_igh_data, rest_stratum)] = 18

In [ ]:
#draw random samples (array indices) from strata
random.seed(123)
sample_pnts_df_all = []


for i in std_df_nona2['stratum']:
    sample_pnts_df = pd.DataFrame()
    print(i)
    hp_igh_data_stratum = np.array(np.where(hp_igh_data==i))
    print("sampling")
    sample_size = int(std_df_nona2.loc[std_df_nona2['stratum'] == i, 'n_samples_incl_min200'].values[0])+100
    print(sample_size)
    sample_columns = random.sample(range(hp_igh_data_stratum.shape[1]), sample_size)
    land_true_indices_sample_ax1 = tuple(hp_igh_data_stratum[0][sample_columns])
    land_true_indices_sample_ax2 = tuple(hp_igh_data_stratum[1][sample_columns])
    sample_pnts_df["stratum"] = hp_igh_data[land_true_indices_sample_ax1, land_true_indices_sample_ax2]
    sample_pnts_df["lat"] = lat_grid[land_true_indices_sample_ax1, land_true_indices_sample_ax2]
    sample_pnts_df["lon"] = lon_grid[land_true_indices_sample_ax1, land_true_indices_sample_ax2]
    sample_pnts_df_all.append(sample_pnts_df)
#     print(sample_pnts_df)
#     print(sample_pnts_df_all)
    
sample_pnts_df_all

In [21]:
df_all = pd.concat(sample_pnts_df_all)

In [22]:
import geopandas
gdf_sample_pnts = geopandas.GeoDataFrame(
    df_all, geometry=geopandas.points_from_xy(x=df_all.lon, y=df_all.lat, crs="+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs")
)

# save samples points to geopackage or reproject first
import pyproj
inProj = '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs'
outProj = '+proj=longlat +datum=WGS84 +no_defs +type=crs'
gdf_sample_pnts_epsg4326 = gdf_sample_pnts.copy()
gdf_sample_pnts_epsg4326
gdf_sample_pnts_epsg4326["lon_epsg4326"], gdf_sample_pnts_epsg4326["lat_epsg4326"] = pyproj.transform(inProj, outProj, gdf_sample_pnts["lon"], gdf_sample_pnts["lat"]) 


tgr_pnts_epsg4326 = geopandas.GeoDataFrame(
    gdf_sample_pnts_epsg4326, geometry=geopandas.points_from_xy(x=gdf_sample_pnts_epsg4326.lon_epsg4326, y=gdf_sample_pnts_epsg4326.lat_epsg4326, crs="+proj=longlat +datum=WGS84 +no_defs +type=crs")
)
tgr_pnts_epsg4326

,stratum,lat,lon,geometry,lon_epsg4326,lat_epsg4326
0,1,6018883.5,3290198.0,POINT (29.30556 54.69267),29.305556,54.692666
1,1,3439670.5,-10092927.0,POINT (-89.12250 30.89909),-89.122495,30.899086
2,1,5531822.0,1173912.0,POINT (1.54535 49.95358),1.545353,49.953582
3,1,1358045.0,3115080.0,POINT (27.93664 12.19953),27.936644,12.199526
4,1,3451829.0,-12216472.0,POINT (-111.36684 31.00831),-111.366841,31.008307
...,...,...,...,...,...,...
2564,18,7462284.5,-9708940.0,POINT (-71.73851 70.29574),-71.738506,70.295744
2565,18,6630795.5,4741220.0,POINT (52.02154 60.95575),52.021539,60.955745
2566,18,2220928.0,5006710.0,POINT (45.93221 19.95094),45.932210,19.950936
2567,18,2282990.0,-730056.0,POINT (-9.03203 20.50845),-9.032034,20.508448


In [23]:
# tgr_pnts_epsg4326.to_file(f'../data/training_pts/trg.pts_strata_{len(tgr_pnts_epsg4326)}_igh.drawn_epsg4326_v20230821.gpkg',  driver="GPKG")